<a href="https://colab.research.google.com/github/Samuelhonorator/Codigo/blob/main/sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este apunte computa el ejemplo 3.3.1 de @daertrycke_risk_2017 donde se tiene un productor neutral al riesgo que resuelve

$$\min_{x,Y}I\cdot x+\tau\mathbb{E}_{\Theta}\left[(C-P_\omega)\cdot Y_\omega\right]\quad\text{subject to}\quad Y_\omega\in[0,x]$$

y la demanda como resultado de resolver

$$\min_Q\tau\mathbb{E}_\Theta\left[P_\omega\cdot Q_\omega-U_\omega(Q_\omega)\right]$$

donde $U_\omega(Q_\omega)=A_\omega\cdot Q_\omega -\frac{B}{2}\cdot Q^2_\omega$

In [ ]:
!pip install gurobipy
from gurobipy import Model, GRB, quicksum

def modeloRNcompetitivecapacityequilibrium(cant_esce, a, theta, B, tau, c, I):
    m = Model()

    # Decision variables x, Y, Q
    x = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name='x')  # Capacidad
    Y = {}  # Producción
    Q = {}  # Demanda

    # Definición de Producción y Demanda para cada estado de la naturaleza Y_omega, Q_omega
    for i in range(cant_esce):
        Y[i] = m.addVar(vtype=GRB.INTEGER, name='Y{}'.format(i))
        Q[i] = m.addVar(vtype=GRB.INTEGER, name='Q{}'.format(i))

    # Restricciones
    for i in range(cant_esce):
        # Production equals demand
        m.addConstr(Y[i] == Q[i])
        # Production bounded by installed capacity
        m.addConstr(Y[i] <= x)

    # F.O
    obj_expr = I * 1000 * x + tau * quicksum(theta[i] * (c * Y[i] - a[i] * Q[i] + (B / 2) * (Q[i] * Q[i])) for i in range(cant_esce))
    m.setObjective(obj_expr, GRB.MINIMIZE)

    m.optimize()

    P = []
    IM = []

    for i in range(cant_esce):
        P.append(a[i] - B * Q[i].x)
        IM.append((tau * (P[i] - c) * Y[i].x - I * x.x) / x.x)

    Qprom = sum(Q[i].x for i in range(cant_esce))

    print("Q Promedio =", Qprom / cant_esce)
    print("x =", x.x)
    print("Valor función objetivo =", m.objVal)

    print(cant_esce, a, theta, B, tau, c, I)

    print("\nQ, Y, P and Investment Margin values:")
    for i in range(cant_esce):
        print("Q({}) = {:.1f}    Y({}) = {:.1f}    P({}) = {:.1f}    Investment Margin = {:.1f}"
          .format(i + 1, Q[i].x, i + 1, Y[i].x, i + 1, P[i], IM[i]))

Parametrizamos una instancia específica del modelo

In [ ]:
cant = 5
A = [300, 350, 400, 450, 500]
Theta = [0.2, 0.2, 0.2, 0.2, 0.2]
B = 1  # $/MWh
tau = 8760  # duracion por segmento por hora
C = 60  # costo de operacion en $/mWhora
I = 90  # capital para gasto

En lo que sigue la resolvemos

In [ ]:
modeloRNcompetitivecapacityequilibrium(cant, A, Theta, B, tau, C, I)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 11 columns and 20 nonzeros
Model fingerprint: 0x0ea22c61
Model has 5 quadratic objective terms
Variable types: 1 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+04, 9e+05]
  QObjective range [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 10 nonzeros
Presolved model has 5 quadratic objective terms
Variable types: 1 continuous, 5 integer (0 binary)
Found heuristic solution: objective -2884020.000

Root relaxation: objective -4.909388e+08, 3 iterat

## Referencias